## Roboticia-quattro : Tutorial 1

### How to move the legs using Pypot Primitives

Instanciate the robot :

In [1]:
from roboticia_quattro import RoboticiaQuattro

In [2]:
robot = RoboticiaQuattro(simulator='vrep')
# if you're lucky enough to have a real robot, just instanciate without any parameters :
# robot = RoboticiaQuattro()

In [ ]:
robot.close()

In [44]:
robot.reset_simulation()

In [4]:
for m in robot.motors :
    m.pid = (8.0,15.0,0.0)

In [4]:
for m in robot.motors :
    print(m.pid)

0
0
0
0
0
0
0
0
0
0
0
0


In [4]:
import math as mt
import time

In [5]:
from pypot.primitive import LoopPrimitive

In [6]:
def AK_side(side_a,side_b,side_c):
    alpha = mt.acos((side_b**2 + side_c**2 - side_a**2)/(2*side_b*side_c))
    beta = mt.acos((side_a**2 + side_c**2 - side_b**2)/(2*side_a*side_c))
    gamma = mt.pi - alpha - beta
    return (alpha,beta,gamma)

def AK_angle(side_a,side_b,gamma):
        side_c = mt.sqrt (side_a**2 + side_b**2 - 2*side_a*side_b*mt.cos(gamma))
        alpha = mt.acos((side_b**2 + side_c**2 - side_a**2)/(2*side_b*side_c))
        beta= mt.pi - alpha - gamma
        return (side_c,alpha,beta)

In [10]:
robot.leg1

[<DxlMotor name=m11 id=11 pos=-0.0>,
 <DxlMotor name=m12 id=12 pos=-0.0>,
 <DxlMotor name=m13 id=13 pos=-0.0>]

In [11]:
class Leg(LoopPrimitive):
    def __init__(self,robot,leg,refresh_freq=50):
        self.robot = robot
        LoopPrimitive.__init__(self, robot,refresh_freq)
        fake_motors = getattr(self.robot, leg)
        self.knee = fake_motors[2]
        self.hip = fake_motors[1]
        self.hip_lateral = fake_motors[0]
        self.motors = fake_motors
        
        # size of segment's leg            
        self.shin = 79
        self.thigh = 72
        self.pelvis = 38
    
    @property      
    def get_pos(self):
        (side_c,alpha,beta) = AK_angle(self.thigh,self.shin,mt.pi-abs(mt.radians(self.knee.present_position)))
        # what knee flexion
        flex = mt.copysign(1,self.knee.present_position)
        # calcul de l'angle beta_2 entre side_c et la veticale
        beta_2 = mt.radians(self.hip.present_position)+beta*flex
        theta = mt.radians(self.hip_lateral.present_position)
        
        high_leg = mt.cos(beta_2)*side_c+self.pelvis
        
        high = mt.cos(theta)*high_leg
        distance = mt.sin(beta_2)*side_c
        balance = mt.sin(theta)*high_leg
        
        return (high,distance,flex,balance)
      
    
    def set_pos(self,h,d,b):
        high_leg = mt.sqrt(h**2+b**2)
        side_c = mt.sqrt((high_leg-self.pelvis)**2+d**2)
        beta_2 = mt.asin(d/side_c)
        (alpha,beta,gamma) = AK_side(self.thigh,self.shin,side_c)
        
        angle_hip = mt.degrees(beta_2 - beta*self.f)
        angle_knee = self.f*mt.degrees(mt.pi - gamma)
        angle_hip_lateral = mt.degrees(mt.asin(b/high_leg))
            
        return(angle_hip_lateral, angle_hip, angle_knee)
        
    def h_limit(self,d):
        pass
        
    def d_limit(self,h):
        pass
        
    def setup(self):
        (self.h,self.d,self.f,self.b) = self.get_pos
        
        
    def update(self):
        (angle_hip_lateral, angle_hip, angle_knee) = self.set_pos(self.h,self.d,self.b)
        self.hip.goal_position = angle_hip
        self.knee.goal_position = angle_knee
        self.hip_lateral.goal_position = angle_hip_lateral
        
    
    def move(self,speed,cycle,*args):
        if cycle == 'go':
            if len(args)>0 and self.d+speed > args[0] : #multiplier par le signe de la vitesse pour gérer les vitesses négatives
                self.d = args[0]
                return True
            else :
                self.d += speed
                return False
            
                     
        if cycle == 'back':
            if self.h-speed > args[0] and self.d-speed > args[1] :
                self.d -= speed
                self.h -= speed
                return False
            
            if self.d-speed > args[1] :
                self.h = args[0]
                self.d -= speed
                return False
            
            if self.h+speed < args[2] : 
                self.d = args[1]
                self.h += speed
                return False
            else :
                self.h = args[2]
                return True
            
                    
        if cycle == 'balance':
            if len(args)>0 and mt.copysign(1,speed)*(self.b+speed) > mt.copysign(1,speed)*args[0] : #multiplier par le signe de la vitesse pour gérer les vitesses négatives
                self.b = args[0]
                return True
            else :
                self.b += speed
                return False
            
        if cycle=='release':
            if self.h-speed < limit : #multiplier par le signe de la vitesse pour gérer les vitesses négatives
                self.h = limit
                return True
            else :
                self.h -= speed
                return False
            
        

In [15]:
leg_rr = Leg(robot,'leg1')

In [17]:
leg_rr.start()

In [21]:
leg_rr.h=170

In [22]:
leg_rl = Leg(robot,'leg4')
leg_fr = Leg(robot,'leg2')
leg_fl = Leg(robot,'leg3')

In [23]:
leg_rl.start()

In [24]:
leg_fr.start()

In [26]:
leg_fl.start()

In [27]:
legs = [leg_rr,leg_fr,leg_fl,leg_rl]

In [29]:
for l in legs:
    l.h=170

In [31]:
leg_rr.f=1

In [32]:
leg_rl.stop()
leg_rr.stop()
leg_fl.stop()
leg_fr.stop()

In [33]:
robot.attach_primitive(Leg(robot,'leg1'), 'leg_rr')

In [39]:
robot.leg_rr.stop()

In [36]:
robot.attach_primitive(Leg(robot,'leg2'), 'leg_fr')

In [37]:
robot.attach_primitive(Leg(robot,'leg3'), 'leg_fl')

In [38]:
robot.attach_primitive(Leg(robot,'leg4'), 'leg_rl')

In [14]:
robot.compliant = False

In [40]:
robot.leg_fr.start()
robot.leg_fl.start()
robot.leg_rr.start()
robot.leg_rl.start()

In [43]:
robot.leg_fr.stop()
robot.leg_fl.stop()
robot.leg_rr.stop()
robot.leg_rl.stop()

In [ ]:
robot.close()

In [16]:
robot.leg_rl.d = 0
robot.leg_fl.d = 0
robot.leg_rr.d = 0
robot.leg_fr.d = 0






In [40]:
robot.leg_rl.b = -20
robot.leg_fl.b = -20
robot.leg_rr.b = 20
robot.leg_fr.b = 20






In [18]:
robot.leg_rl.b, robot.leg_rr.b, robot.leg_fl.b, robot.leg_fr.b

(-20, 20, -20, 20)

In [18]:
for i in range(80):
    robot.leg_fr.h = 80 + i
    robot.leg_fl.h = 80 + i
    robot.leg_rr.h = 80 + i
    robot.leg_rl.h = 80 + i
    time.sleep(0.02)

In [56]:
for m in robot.motors :
    print(m.name, m.present_load,m.present_temperature)

m11 3.9 31.0
m12 -4.6 32.0
m13 -12.4 37.0
m21 18.2 32.0
m22 9.6 32.0
m23 4.6 33.0
m31 -4.8 32.0
m32 -11.0 31.0
m33 4.2 35.0
m41 14.4 33.0
m42 -4.2 32.0
m43 -32.6 37.0


In [20]:
robot.leg_fr.h = 140
robot.leg_fr.d = 30
robot.leg_rl.h = 140
robot.leg_rl.d = 10
time.sleep(0.1)
robot.leg_rl.h = 160
robot.leg_fr.h = 160


In [21]:
robot.leg_fl.h = 140
robot.leg_fl.d = -10
robot.leg_rr.h = 140
robot.leg_rr.d = -30
time.sleep(0.1)
robot.leg_fl.h = 160
robot.leg_rr.h = 160

In [72]:
robot.leg_fr.h = 160
robot.leg_fl.h = 160
robot.leg_rr.h = 160
robot.leg_rl.h = 160

In [19]:
robot.leg_fr.stop()
robot.leg_fl.stop()
robot.leg_rr.stop()
robot.leg_rl.stop()

In [102]:
c = [False] *8
while not (c[0]&c[1]&c[2]&c[3]&c[4]&c[5]&c[6]&c[7]):
    c[0] = robot.leg_fl.move(1,'balance',5)
    c[1] = robot.leg_fr.move(1,'balance',45)
    c[2] = robot.leg_rr.move(1,'balance',45)
    c[3] = robot.leg_rl.move(1,'balance',5)
    
    #c[4] = robot.leg_fr.move(1,'go',30)
    c[5] = robot.leg_rl.move(1,'go',20)
    c[6] = robot.leg_fl.move(1,'go',10)
    c[7] = robot.leg_rr.move(1,'go',-20)
    
    c[4] = robot.leg_fr.move(1,'back',140,-50,160)
    time.sleep(0.02)

In [103]:
c = [False] *8
while not (c[0]&c[1]&c[2]&c[3]&c[4]&c[5]&c[6]&c[7]):
    c[0] = robot.leg_fl.move(-1,'balance',-45)
    c[1] = robot.leg_fr.move(-1,'balance',-5)
    c[2] = robot.leg_rr.move(-1,'balance',-5)
    c[3] = robot.leg_rl.move(-1,'balance',-45)
    
    c[4] = robot.leg_fr.move(1,'go',-20)
    c[5] = robot.leg_rl.move(1,'go',50)
    c[6] = robot.leg_fl.move(1,'go',40)
    c[7] = robot.leg_rr.move(1,'go',10)
    time.sleep(0.02)

In [104]:
while not robot.leg_rl.move(1,'back',140,-50,160) : time.sleep(0.02)

In [105]:
while not robot.leg_fl.move(1,'back',140,-50,160) : time.sleep(0.02)

In [106]:
c = [False] *8
while not (c[0]&c[1]&c[2]&c[3]&c[4]&c[5]&c[6]&c[7]):
    c[0] = robot.leg_fl.move(1,'balance',5)
    c[1] = robot.leg_fr.move(1,'balance',45)
    c[2] = robot.leg_rr.move(1,'balance',45)
    c[3] = robot.leg_rl.move(1,'balance',5)
    
    c[4] = robot.leg_fr.move(1,'go',20)
    c[5] = robot.leg_rl.move(1,'go',-10)
    c[6] = robot.leg_fl.move(1,'go',-20)
    c[7] = robot.leg_rr.move(1,'go',40)
    time.sleep(0.02)

In [107]:
while not robot.leg_rr.move(1,'back',140,-50,160) : time.sleep(0.02)

In [110]:
for i in range(10):
    c = [False] *8
    while not (c[0]&c[1]&c[2]&c[3]&c[4]&c[5]&c[6]&c[7]):
        c[0] = robot.leg_fl.move(1,'balance',5)
        c[1] = robot.leg_fr.move(1,'balance',45)
        c[2] = robot.leg_rr.move(1,'balance',45)
        c[3] = robot.leg_rl.move(1,'balance',5)

        #c[4] = robot.leg_fr.move(1,'go',30)
        c[5] = robot.leg_rl.move(1,'go',20)
        c[6] = robot.leg_fl.move(1,'go',10)
        c[7] = robot.leg_rr.move(1,'go',-20)

        c[4] = robot.leg_fr.move(1,'back',140,-50,160)
        time.sleep(0.02)

    c = [False] *8
    while not (c[0]&c[1]&c[2]&c[3]&c[4]&c[5]&c[6]&c[7]):
        c[0] = robot.leg_fl.move(-1,'balance',-45)
        c[1] = robot.leg_fr.move(-1,'balance',-5)
        c[2] = robot.leg_rr.move(-1,'balance',-5)
        c[3] = robot.leg_rl.move(-1,'balance',-45)

        c[4] = robot.leg_fr.move(1,'go',-20)
        c[5] = robot.leg_rl.move(1,'go',50)
        c[6] = robot.leg_fl.move(1,'go',40)
        c[7] = robot.leg_rr.move(1,'go',10)
        time.sleep(0.02)

    while not robot.leg_rl.move(1,'back',140,-50,160) : time.sleep(0.02)

    while not robot.leg_fl.move(1,'back',140,-50,160) : time.sleep(0.02)

    c = [False] *8
    while not (c[0]&c[1]&c[2]&c[3]&c[4]&c[5]&c[6]&c[7]):
        c[0] = robot.leg_fl.move(1,'balance',5)
        c[1] = robot.leg_fr.move(1,'balance',45)
        c[2] = robot.leg_rr.move(1,'balance',45)
        c[3] = robot.leg_rl.move(1,'balance',5)

        c[4] = robot.leg_fr.move(1,'go',20)
        c[5] = robot.leg_rl.move(1,'go',-10)
        c[6] = robot.leg_fl.move(1,'go',-20)
        c[7] = robot.leg_rr.move(1,'go',40)
        time.sleep(0.02)

    while not robot.leg_rr.move(1,'back',140,-50,160) : time.sleep(0.02)

In [71]:
robot.leg_rr.h = 160

In [42]:
class Walk(LoopPrimitive):
    
    def setup(self):
              
        #set the walk variable
        self.go = True
        self.speed = 1
        self.new_speed = 1
        self.mini = -10
        self.maxi = 50
        self.high = 20
        self.shape = 0.3
        self.h = 100
        self.balance = 0
        
        #set the speed for movement
        for m in robot.motors : m.moving_speed = 50
        
        #put the leg verticale
        robot.leg_rl.b = -20
        robot.leg_fl.b = -20
        robot.leg_rr.b = 20
        robot.leg_fr.b = 20
        time.sleep(2)
                
        #stand up the robot
        robot.leg_fr.h = 150
        robot.leg_fl.h = 150
        robot.leg_rr.h = 150
        robot.leg_rl.h = 150
        time.sleep(2)
                
        #set the leg to be ready to start the walk
        for m in robot.motors : m.moving_speed = 0
        robot.leg_fr.h = 80
        robot.leg_fr.d = 50
        time.sleep(0.1)
        robot.leg_fr.h = 100
        print('leg1')
        time.sleep(3)
        
        
        robot.leg_rr.h = 80
        robot.leg_rr.d = -10
        time.sleep(0.1)
        robot.leg_rr.h = 100
        print('leg2')
        time.sleep(3)
        
        
        for m in robot.motors : m.moving_speed = 50
        robot.leg_fr.b = -3
        robot.leg_fl.b = -3
        robot.leg_rr.b = -3
        robot.leg_rl.b = -3
        time.sleep(3)
    
        for m in robot.motors : m.moving_speed = 0
        robot.leg_rl.h = 80
        robot.leg_rl.d = 30
        time.sleep(0.1)
        robot.leg_rl.h = 100
        print('leg3')
        time.sleep(3)
        
        robot.leg_fl.h = 80
        robot.leg_fl.d = 10
        time.sleep(0.1)
        robot.leg_fl.h = 100
        print('leg4')
        time.sleep(3)
        
        for m in robot.motors : m.moving_speed = 50
        robot.leg_fr.b = 0
        robot.leg_fl.b = 0
        robot.leg_rr.b = 0
        robot.leg_rl.b = 0
        time.sleep(3)
        
        for m in robot.motors : m.moving_speed = 0
        robot.leg_fr.go = False
        robot.leg_fr.return_pos = self.step_back(robot.leg_fr.d)
        print(robot.leg_fr.return_pos)
        robot.leg_fl.go = True
        robot.leg_rr.go = True
        robot.leg_rl.go = True
      
       
    
    def step_back(self,pos_d):
        h = self.h
        mini = self.mini
        high = self.high
        shape = self.shape
        speed = self.speed
        step = [i for i in range(pos_d-3*speed,mini-3*speed,-3*speed)]
        step_up = round(len(step)*shape)
        step_down = len(step) - step_up
        step_list_up = [h-(i+1)*(high/step_up) for i in range(0,step_up)]
        step_list_down = [h-i*(high/step_down) for i in range(step_down-1,-1,-1)]
        step_list = step_list_up + step_list_down
        step_dict = dict(zip(step,step_list))
        return step_dict
    
    def update(self):
        
               
        if robot.leg_fr.d <= self.mini and not robot.leg_fr.go :
            robot.leg_fr.go = not robot.leg_fr.go
            robot.leg_rl.go = not robot.leg_rl.go
            robot.leg_rl.return_pos = self.step_back(robot.leg_rl.d)
            self.speed = self.new_speed
            
        if robot.leg_rl.d <= self.mini and not robot.leg_rl.go :
            robot.leg_rl.go = not robot.leg_rl.go
            robot.leg_fl.go = not robot.leg_fl.go
            robot.leg_fl.return_pos = self.step_back(robot.leg_fl.d)
            self.speed = self.new_speed
            
        if robot.leg_fl.d <= self.mini and not robot.leg_fl.go :
            robot.leg_fl.go = not robot.leg_fl.go
            robot.leg_rr.go = not robot.leg_rr.go
            robot.leg_rr.return_pos = self.step_back(robot.leg_rr.d)
            self.speed = self.new_speed
            
        if robot.leg_rr.d <= self.mini and not robot.leg_rr.go :
            robot.leg_rr.go = not robot.leg_rr.go
            robot.leg_fr.go = not robot.leg_fr.go
            robot.leg_fr.return_pos = self.step_back(robot.leg_fr.d)
            self.speed = self.new_speed
            
        # robot leg is forward
        if robot.leg_fr.go : robot.leg_fr.d += self.speed
        else : 
            # we need to use the return_pos dictionnary to have the high of the leg
            robot.leg_fr.d += -3*self.speed
            robot.leg_fr.h = robot.leg_fr.return_pos[robot.leg_fr.d]
            #print('return : d = %f and h = %f' %  (robot.leg_fr.d,robot.leg_fr.h))
        
        if robot.leg_fl.go : robot.leg_fl.d += self.speed
        else : 
            # we need to use the return_pos dictionnary to have the high of the leg
            robot.leg_fl.d += -3*self.speed
            robot.leg_fl.h = robot.leg_fl.return_pos[robot.leg_fl.d]
            #print('return : d = %f and h = %f' %  (robot.leg_fr.d,robot.leg_fr.h))
            
        if robot.leg_rr.go : robot.leg_rr.d += self.speed
        else : 
            # we need to use the return_pos dictionnary to have the high of the leg
            robot.leg_rr.d += -3*self.speed
            robot.leg_rr.h = robot.leg_rr.return_pos[robot.leg_rr.d]
            #print('return : d = %f and h = %f' %  (robot.leg_fr.d,robot.leg_fr.h))
            
        if robot.leg_rl.go : robot.leg_rl.d += self.speed
        else : 
            # we need to use the return_pos dictionnary to have the high of the leg
            robot.leg_rl.d += -3*self.speed
            robot.leg_rl.h = robot.leg_rl.return_pos[robot.leg_rl.d]
            #print('return : d = %f and h = %f' %  (robot.leg_fr.d,robot.leg_fr.h))
        #print('go : d = %d' %  (robot.leg_fr.d))
        
        #robot leg is backward
        
                        

In [34]:
walk = Walk(robot,50)

In [26]:
set_actual_pos(robot)

In [35]:
walk.start()

leg1
leg2
leg3
leg4


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/lib/python3.4/threading.py", line 920, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.4/threading.py", line 868, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.4/dist-packages/pypot/utils/stoppablethread.py", line 121, in _wrapped_target
    self._setup()
  File "/usr/local/lib/python3.4/dist-packages/pypot/primitive/primitive.py", line 67, in _prim_setup
    self.setup()
  File "<ipython-input-33-a5ee8a90684d>", line 89, in setup
    robot.leg_fr.return_pos = self.step_back(robot.leg_fr.d)
  File "<ipython-input-33-a5ee8a90684d>", line 103, in step_back
    step = [i for i in range(pos_d-3*speed,mini-3*speed,-3*speed)]
TypeError: 'float' object cannot be interpreted as an integer



RuntimeError: Setup failed, see Thread-18 Tracebackfor details.

In [36]:
walk.stop()

In [298]:
walk.new_speed = 1

In [111]:
robot.compliant = True

In [112]:
robot.close()

In [245]:
primitive_list = [robot.leg_fr,robot.leg_rr,robot.leg_fl,robot.leg_rl]

In [246]:
for p in primitive_list :
    p.d = p.get_pos[1]
    p.h = p.get_pos[0]

In [27]:
robot.compliant = False

In [64]:
robot.m11.moving_speed

80

In [39]:
for m in robot.leg1 : m.compliant = True

In [197]:
robot.leg_fr.b = 0
robot.leg_fl.b = 0
robot.leg_rr.b = 0
robot.leg_rl.b = 0

In [198]:
for m in robot.motors : print(m.present_temperature)

32.0
32.0
43.0
31.0
32.0
35.0
32.0
32.0
43.0
32.0
31.0
38.0


In [65]:
motors_hip_lateral = [robot.m11,robot.m21,robot.m31,robot.m41]

In [70]:
for m in motors_hip_lateral : m.moving_speed = 10

In [119]:
for m in motors_hip_lateral : m.goal_position = 0

In [137]:
robot.m11.goal_position = -5
robot.m31.goal_position = -0
robot.m21.goal_position = -5
robot.m41.goal_position = -0


In [108]:
for m in robot.leg1 : m.moving_speed = 0

In [136]:
robot.leg_rl.h = 80
time.sleep(0.2)
robot.leg_rl.h = 100

In [114]:
print(robot.leg_rr.h,robot.leg_rl.h,robot.leg_fr.h,robot.leg_fl.h)

100 100 100.0 100


In [105]:
def set_actual_pos(robot):
    for m in robot.motors : 
        m.goal_position = m.present_position
    robot.compliant = False

In [21]:
robot._primitive_manager.__dict__

{'_crashed': False,
 '_filter': functools.partial(<function mean at 0x714234b0>, axis=0),
 '_motors': [<DxlMotor name=m11 id=11 pos=-0.44>,
  <DxlMotor name=m12 id=12 pos=51.47>,
  <DxlMotor name=m13 id=13 pos=-139.44>,
  <DxlMotor name=m21 id=21 pos=3.37>,
  <DxlMotor name=m22 id=22 pos=59.97>,
  <DxlMotor name=m23 id=23 pos=-143.84>,
  <DxlMotor name=m31 id=31 pos=-0.15>,
  <DxlMotor name=m32 id=32 pos=58.8>,
  <DxlMotor name=m33 id=33 pos=-145.01>,
  <DxlMotor name=m41 id=41 pos=-0.73>,
  <DxlMotor name=m42 id=42 pos=54.4>,
  <DxlMotor name=m43 id=43 pos=-137.1>],
 '_prim': [<__main__.Leg at 0x6db39810>,
 '_resume': <threading.Event at 0x6e3e2490>,
 '_running': <threading.Event at 0x6e3e2450>,
 '_setup': <bound method PrimitiveManager.setup of <pypot.primitive.manager.PrimitiveManager object at 0x6e3e23f0>>,
 '_started': <threading.Event at 0x6e3e2410>,
 '_target': <bound method PrimitiveManager.run of <pypot.primitive.manager.PrimitiveManager object at 0x6e3e23f0>>,
 '_teardown': <

In [49]:
dxl_io=robot._controllers[0]

In [50]:
dir(dxl_io)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_crashed',
 '_resume',
 '_running',
 '_setup',
 '_started',
 '_target',
 '_teardown',
 '_thread',
 '_update',
 '_updated',
 '_wrapped_target',
 'close',
 'controllers',
 'io',
 'join',
 'motors',
 'pause',
 'paused',
 'period',
 'resume',
 'run',
 'running',
 'setup',
 'should_pause',
 'should_stop',
 'start',
 'started',
 'stop',
 'teardown',
 'update',
 'wait_to_resume',
 'wait_to_start',
 'wait_to_stop']

In [344]:
robot.m32.led = 'green'

In [349]:
robot.m31.led = 'off'

In [353]:
robot.m31.present_temperature

28.0

In [357]:
dir(dxl_io)

['_AbstractDxlIO__controls',
 '_AbstractDxlIO__force_lock',
 '_AbstractDxlIO__real_read',
 '_AbstractDxlIO__real_send',
 '_AbstractDxlIO__used_ports',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_change_baudrate',
 '_change_id',
 '_convert',
 '_error_handler',
 '_generate_accessors',
 '_get_control_value',
 '_get_goal_pos_speed',
 '_get_model',
 '_get_pid_gain',
 '_get_status_return_level',
 '_known_mode',
 '_known_models',
 '_open',
 '_protocol',
 '_send_packet',
 '_serial',
 '_serial_lock',
 '_set_LED',
 '_set_control_value',
 '_set_goal_pos_speed',
 '_set_pid_gain',
 '_set_status_return_level',
 '_set_torque_enable',
 '_sync_read',
 'baudrate',

In [374]:
dxl_io.set_LED_color({33:'green'})

In [403]:
dxl_io.switch_led_off({33})

In [402]:
dxl_io.set_alarm_shutdown({33:['Overheating Error']})

In [405]:
dxl_io.get_alarm_shutdown({33})

(('Overheating Error',),)

In [404]:
dxl_io.get_alarm_shutdown({32})

(('Angle Limit Error', 'Input Voltage Error'),)

In [66]:
for i in range(10) : print(i)

0
1
2
3
4
5
6
7
8
9


In [77]:


@timeit
def test(*args):
    if len(args)>0 and args[0] > 10: print(args[0])

In [78]:

test()
print('coucou')

'test' ((), {}) 0.00 sec
coucou


In [75]:
import time                                                

def timeit(method):

    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        print('%r (%r, %r) %2.2f sec' % (method.__name__, args, kw, te-ts))
        return result

    return timed

In [34]:
True & True & True & True

True

In [38]:
True and True

True

In [ ]:
while True :
    